In [1]:
import os
os.environ['HF_HOME'] = '/run/cache/'

In [2]:
# Load the dataset
from datasets import load_dataset

dataset = load_dataset('Amod/mental_health_counseling_conversations', split='train')

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [3]:
df = dataset.to_pandas()

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

config.json:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [5]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   36G   65G  36% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   36G   65G  36% /run
tmpfs            14G  4.0K   14G   1% /dev/shm
/dev/nvme2n1    2.0G  1.9G     0 100% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G  4.7M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [27]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)


generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.9,  # Increase temperature for more creativity
    "top_k": 50,         # Limits the sampling pool to the top 50 tokens
    "do_sample": True,  
}


In [16]:
df1 = dataset.to_pandas()
df1.head(1)

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."


In [29]:
from collections import defaultdict

def generate_questions(sample_df, num_questions=2):
    all_questions = []

    for _, row in sample_df.iterrows():
        context = row['Context']
        response = row['Response']
        
        # Prepare the prompt
        prompt = f"""
        You are emulating a person experiencing mental health challenges.
        Formulate {num_questions*2} insightful questions based on the following mental health conversation record. 
        Use the context provided to generate relevant questions, but avoid directly copying words from the response.

        Record:
        Context: {context}
        Response: {response}

        Provide the output in the format:
        "Question 1: Question text"
        "Question 2: Question text"
        ...
        """

        # Generate the questions
        result = pipe(prompt, **generation_args)[0]["generated_text"]

        # Extract questions
        questions = []
        for line in result.split('\n'):
            if 'Question' in line:
                parts = line.split(":", 1)
                if len(parts) > 1:
                    question_text = parts[1].strip()
                    if question_text:
                        questions.append(question_text)
        
        # Score and select top num_questions questions
        if len(questions) > num_questions:
            scores = score_questions(questions, context)
            sorted_questions = [q for _, q in sorted(zip(scores, questions), reverse=True)]
            selected_questions = sorted_questions[:num_questions]
        else:
            selected_questions = list(dict.fromkeys(questions))  # Remove duplicates if less than num_questions
        
        # Add the questions to the list with their corresponding context
        all_questions.append({
            'Context': context,
            'Questions': selected_questions
        })
    
    return all_questions

# Example usage with DataFrame
sample_df = df1.sample(n=3, random_state=42)
questions_list = generate_questions(sample_df)

Context: I've hit my head on walls and floors ever since I was young. I sometimes still do it but I don't exactly know why,    I have anxiety and I had a rough childhood but now I'll start to hit my head and sometimes not realize it but I don't know how to stop or even why I'm doing it.    How can I help myself to change my behavior?
Question 1: What resources or support systems are available for someone like me, who is recognizing their mental health needs but uncertain about how to proceed with changing harmful habits?"
Question 2: Given that my childhood experiences might be influencing my current behavior, how can I seek therapy that acknowledges and treats trauma differently than general anxiety therapy?"

Context: Over a year ago I had a female friend. She turned out to be kind of crazy so I decided to stop talking to her. When she would call me I wouldn't answer the phone. This made my girlfriend really suspicious. She would ask me why I wouldn't ever answer that phone number. I

In [31]:
from collections import defaultdict

def generate_questions(sample_df, num_questions=2):
    all_questions = []

    for _, row in sample_df.iterrows():
        context = row['Context']
        response = row['Response']
        
        # Prepare the prompt
        prompt = f"""
        You are emulating a person experiencing mental health challenges.
        Formulate {num_questions*2} insightful questions based on the following mental health conversation record. 
        Use the context provided to generate relevant questions, but avoid directly copying words from the response.

        Record:
        Context: {context}
        Response: {response}

        Provide the output in the format:
        "Question 1: Question text"
        "Question 2: Question text"
        ...
        """

        # Generate the questions
        result = pipe(prompt, **generation_args)[0]["generated_text"]

        # Extract questions
        questions = []
        for line in result.split('\n'):
            if 'Question' in line:
                parts = line.split(":", 1)
                if len(parts) > 1:
                    question_text = parts[1].strip()
                    if question_text:
                        questions.append(question_text)
        
        # Score and select top num_questions questions
        if len(questions) > num_questions:
            scores = score_questions(questions, context)
            sorted_questions = [q for _, q in sorted(zip(scores, questions), reverse=True)]
            selected_questions = sorted_questions[:num_questions]
        else:
            selected_questions = list(dict.fromkeys(questions))  # Remove duplicates if less than num_questions
        
        # Add the questions to the list with their corresponding context
        all_questions.append({
            'Context': context,
            'Questions': selected_questions
        })
    
    return all_questions

# Example usage with DataFrame
sample_df = df1.sample(n=500, random_state=42)

# questions_list = generate_questions(sample_df)
# for record in questions_list:
#     print(f"Context: {record['Context']}")
#     for i, question in enumerate(record['Questions'], 1):
#         print(f"Question {i}: {question}")
#     print()

In [32]:
questions_list = generate_questions(sample_df)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [33]:
# Print the first few records to inspect the output
for i, record in enumerate(questions_list[:5]):  # Adjust the number as needed
    print(f"Context {i + 1}: {record['Context']}")
    for j, question in enumerate(record['Questions'], 1):
        print(f"  Question {j}: {question}")
    print()  # For spacing between different records


Context 1: I've hit my head on walls and floors ever since I was young. I sometimes still do it but I don't exactly know why,    I have anxiety and I had a rough childhood but now I'll start to hit my head and sometimes not realize it but I don't know how to stop or even why I'm doing it.    How can I help myself to change my behavior?

Context 2: Over a year ago I had a female friend. She turned out to be kind of crazy so I decided to stop talking to her. When she would call me I wouldn't answer the phone. This made my girlfriend really suspicious. She would ask me why I wouldn't ever answer that phone number. I told my girlfriend that I don't want to be friends with that other woman, but I don't think she believes me. How can I get my girlfriend to understand?

Context 3: My long-distance girlfriend is in a sorority, and it's changing her. I feel like I'm becoming less important to her and it hurts. She just wants me to support the sorority, but it's so hard. I try every day to show 

In [35]:
import copy

# Assuming questions_list is your original list
questions_list_copy = copy.deepcopy(questions_list)


In [34]:
# Calculate the total number of questions generated
total_questions = sum(len(record['Questions']) for record in questions_list)
print(f"Total number of generated questions: {total_questions}")


Total number of generated questions: 860


In [36]:
import uuid

def add_ids_to_questions_list(questions_list_copy):
    # Initialize a new list to store results with IDs
    questions_with_ids = []

    # Iterate through the questions list and assign unique IDs
    for record in questions_list_copy:
        context = record['Context']
        questions = record['Questions']
        
        # Generate a unique ID for this context and associated questions
        unique_id = uuid.uuid4().hex[:10]  # Generate a shorter unique ID
        
        # Add ID to each record and the questions
        questions_with_ids.append({
            'ID': unique_id,
            'Context': context,
            'Questions': [{'ID': unique_id, 'Question': question} for question in questions]
        })

    return questions_with_ids

# Apply the ID assignment function
questions_list_with_ids = add_ids_to_questions_list(questions_list_copy)

# Example output inspection
for record in questions_list_with_ids[:5]:  # Adjust the number to view more records
    print(f"ID: {record['ID']}")
    print(f"Context: {record['Context']}")
    for question in record['Questions']:
        print(f"  QuestionID: {question['ID']}, Question: {question['Question']}")
    print()  # For spacing between records


ID: 82b18f1d35
Context: I've hit my head on walls and floors ever since I was young. I sometimes still do it but I don't exactly know why,    I have anxiety and I had a rough childhood but now I'll start to hit my head and sometimes not realize it but I don't know how to stop or even why I'm doing it.    How can I help myself to change my behavior?

ID: b9ea4fc5fa
Context: Over a year ago I had a female friend. She turned out to be kind of crazy so I decided to stop talking to her. When she would call me I wouldn't answer the phone. This made my girlfriend really suspicious. She would ask me why I wouldn't ever answer that phone number. I told my girlfriend that I don't want to be friends with that other woman, but I don't think she believes me. How can I get my girlfriend to understand?

ID: f307b325d9
Context: My long-distance girlfriend is in a sorority, and it's changing her. I feel like I'm becoming less important to her and it hurts. She just wants me to support the sorority, but

In [37]:
import pandas as pd

# Flatten the questions_list_with_ids to create the DataFrame
def flatten_questions(questions_list_with_ids):
    # Initialize lists to hold data for the DataFrame
    question_texts = []
    document_ids = []
    
    for record in questions_list_with_ids:
        unique_id = record['ID']
        for question in record['Questions']:
            question_texts.append(question['Question'])
            document_ids.append(unique_id)
    
    # Create the DataFrame
    df2 = pd.DataFrame({
        'questions': question_texts,
        'documents': document_ids
    })
    
    return df2

# Apply the flattening function
questions_df = flatten_questions(questions_list_with_ids)

In [38]:
print(questions_df.shape)  # Should be (860, 2)

(860, 2)


In [40]:
questions_df.head(10)

,questions,documents
0,How do you both feel about seeking external su...,f307b325d9
1,Have you previously used any form of communica...,f307b325d9
2,What steps can you take to ensure that your ow...,842e439c52
3,How can setting personal boundaries and practi...,842e439c52
4,In what ways have your past experiences shaped...,af8cba43be
5,What are some moments or activities that bring...,af8cba43be
6,"Looking forward, what qualities do you hope to...",8c303a7fdc
7,Reflecting on your own feelings during the bre...,8c303a7fdc
8,"Considering the influence of cognitive biases,...",b1023f460a
9,Reflecting on the role of emotional intelligen...,b1023f460a


In [42]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   43G   58G  43% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   43G   58G  43% /run
tmpfs            14G  4.0K   14G   1% /dev/shm
/dev/nvme2n1    2.0G  1.9G     0 100% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G   19M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [45]:
# Move the large incomplete Hugging Face cache file to /run/cache/
!mv /home/jovyan/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct/blobs/fc534bed68dc12e48d1f28d144bfb2239bd1ef4f5d5892d68e9d1f85731b8c12.incomplete /run/cache/

# Move the entire Hugging Face cache directory for the specific model to /run/cache/
!mv /home/jovyan/.cache/huggingface/hub/models--microsoft--Phi-3-mini-128k-instruct /run/cache/

# Move the pip cache directory to /run/cache/
!mv /home/jovyan/.cache/pip /run/cache/


In [46]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   45G   56G  45% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   45G   56G  45% /run
tmpfs            14G  4.0K   14G   1% /dev/shm
/dev/nvme2n1    2.0G  9.9M  1.9G   1% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G   20M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [47]:
import pickle
questions_df.to_pickle('questions_documents.pkl')

In [48]:
import pandas as pd

# Load the DataFrame from the pickle file
loaded_df = pd.read_pickle('questions_documents.pkl')

# Check the shape and a few rows to verify
print(loaded_df.shape)  # Should match the shape of the original DataFrame
# print(loaded_df.head())


(860, 2)


In [49]:
loaded_df.head()

,questions,documents
0,How do you both feel about seeking external su...,f307b325d9
1,Have you previously used any form of communica...,f307b325d9
2,What steps can you take to ensure that your ow...,842e439c52
3,How can setting personal boundaries and practi...,842e439c52
4,In what ways have your past experiences shaped...,af8cba43be
